# 小环境/猪群舒适度指标体系
## 第一阶段：
#### 环控参数-->猪群舒适度（处方笺咳嗽/腹泻）

从温度、湿度、喷淋、风量等环境参数中，结合猪群自身因素和时间变量，总结提取影响猪群健康和舒适度（造成咳嗽/腹泻）的关键环境因素。

## 星环/环控数据库取数、数据聚合 （时间较久，一个月数据大概需要8小时）

In [ ]:
# import sys
# import re
# from pyhive import hive
# import pandas as pd
# import numpy as np
# import requests
# import json
# import warnings
# warnings.filterwarnings('ignore')
# sys.path.append('/data/datamining/envir/Py_functions')
# from get_data.get_data_func import * ##取数function
# ## 保育信息表去批次信息
# df = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
#            sql = '''SELECT BatchNO,InDate,OutDate,InGoodQty,InBadQty,OutQty,OutBadQty,InDayOld,OutDayOld,StarAvgWeight,EndAvgWeight   
#            FROM mydw.source_my_sc_consalary 
#            where date(outdate) between '20201001' and '20201030' ''').drop_duplicates()
# batchlist = tuple(df.batchno.unique())

# raw_field = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
#            sql = '''SELECT fbatchno,ffieldid,funitnames as unit
#            FROM mydw.source_my_sc_pigbatch 
#            where fbatchno in ''' + str(batchlist))

# def expand_unit(df):  # 批次信息的处理 —— 1.识别单元号 2.将不同单元拆分成不同的记录
#     df['unit'] = df.unit.apply(lambda x: re.findall(r'\d+', x))
#     res = []
#     for i in range(df.shape[0]):
#         temp = df.iloc[i,:]
#         for j in temp['unit']:
#             cand = dict(temp)
#             cand['unit'] = j
#             res.append(cand)
#     return pd.DataFrame(res)
# field = expand_unit(raw_field)

# batch_info = pd.merge(field,df,left_on = 'fbatchno', right_on = 'batchno', how = 'outer').drop('fbatchno',axis = 1)
# unit = pd.read_csv('unit.csv').rename({'field_id':'ffieldid'},axis = 1).astype({'unit':'str'})
# step_2_data = unit.merge(batch_info,on = ['ffieldid','unit'],how = 'inner').astype({'indate':'datetime64','outdate':'datetime64'})

# def data_mani(dic,size = '10min'): # 按照基础信息，匹配环控数据并对其进行数据处理
#     env_data = search_data(UnitID = dic['id'],Size = 999999,StartTime = str(dic['indate']), LimitTime = str(dic['outdate'])) 
    
#     #读取环控数据，若该单元没有环控数据，则返回None
#     try:
#         if env_data.shape[0] == 0: 
#             print('没有环控数据 : ' +dic['id'] )
#             return None
#     except:
#         print("环控数据报错: "+dic['id'])
#         return None
#     env_data = env_data.applymap(lambda x: 0 if x =='' else x).fillna(method = 'ffill')
#     env_data = env_data.astype({'time':'datetime64','shower_1_acc_uptime':'int','shower_2_acc_uptime':'int'}).set_index('time')
    
#     #temperature：室内两个温度传感器的平均值，然后以size为最小时间粒度做聚合求平均，默认10min
#     raw_temp = env_data[['temperature_inner_1','temperature_inner_2']]
#     raw_temp['temperature_inner_1'] = raw_temp.apply(lambda x: x.temperature_inner_2 if (x.temperature_inner_1<5)|(x.temperature_inner_1>40) 
#                                                      else x.temperature_inner_1, axis = 1)
#     raw_temp = raw_temp[(raw_temp.temperature_inner_1 >0)&(raw_temp.temperature_inner_1 <40)]
#     raw_temp['temperature_inner_2'] = raw_temp.apply(lambda x: x.temperature_inner_1 if (x.temperature_inner_2<5)|(x.temperature_inner_2>40)
#                                                      else x.temperature_inner_2, axis = 1)
#     temperature = raw_temp.resample(size).mean().apply(lambda x: x.temperature_inner_1/2+x.temperature_inner_2/2,axis = 1)    
    
#     #shower：室内两个喷淋设备的开启时间，以size为最小单位时间，两个设备单位时间内开启的累计时长加和，size默认10min
#     shower = (env_data.resample(size).max() - env_data.resample(size).min()).apply(lambda x: x.shower_1_acc_uptime + x.shower_2_acc_uptime, axis = 1 )

#     #区分日间夜间，早8-晚8为日间，此外为夜间。 温度和喷淋均区分日间夜间情况
#     raw_temp_day = temperature[(temperature.index.hour >=8) & (temperature.index.hour<20)]
#     raw_temp_night = temperature[(temperature.index.hour <8) | (temperature.index.hour>=20)]
#     raw_shower_day = shower[(shower.index.hour >=8) & (shower.index.hour < 20)]
#     raw_shower_night = shower[(shower.index.hour <8) | (shower.index.hour>=20)]
#     diminished_data = []
    
#     #分别计算日/夜间的温度/喷淋数据的四个指标：max,min,mean,std
#     diminished_data.append(raw_temp_day.groupby(raw_temp_day.index.date).quantile(0.9))
#     diminished_data.append(raw_temp_day.groupby(raw_temp_day.index.date).quantile(0.1))
#     diminished_data.append(raw_temp_day.groupby(raw_temp_day.index.date).mean())
#     diminished_data.append(raw_temp_day.groupby(raw_temp_day.index.date).std())
#     diminished_data.append(raw_temp_night.groupby(raw_temp_night.index.date).quantile(0.9))
#     diminished_data.append(raw_temp_night.groupby(raw_temp_night.index.date).quantile(0.1))
#     diminished_data.append(raw_temp_night.groupby(raw_temp_night.index.date).mean())
#     diminished_data.append(raw_temp_night.groupby(raw_temp_night.index.date).std())
#     diminished_data.append(raw_shower_day.groupby(raw_shower_day.index.date).quantile(0.9))
#     diminished_data.append(raw_shower_day.groupby(raw_shower_day.index.date).quantile(0.1))
#     diminished_data.append(raw_shower_day.groupby(raw_shower_day.index.date).mean())
#     diminished_data.append(raw_shower_day.groupby(raw_shower_day.index.date).std())
#     diminished_data.append(raw_shower_night.groupby(raw_shower_night.index.date).quantile(0.9))
#     diminished_data.append(raw_shower_night.groupby(raw_shower_night.index.date).quantile(0.1))
#     diminished_data.append(raw_shower_night.groupby(raw_shower_night.index.date).mean())
#     diminished_data.append(raw_shower_night.groupby(raw_shower_night.index.date).std())
#     diminished_data = pd.concat(diminished_data,axis = 1).reset_index()
#     var_name = ['day_temp_max','day_temp_min','day_temp_mean','day_temp_std',
#                               'night_temp_max','night_temp_min','night_temp_mean','night_temp_std',
#                               'day_shower_max','day_shower_min','day_shower_mean','day_shower_std',
#                               'night_shower_max','night_shower_min','night_shower_mean','night_shower_std']
#     diminished_data.columns = ['time'] + var_name
    
#     #添加基础信息，包括批次号、日龄、厂区号、单元、转入猪头数、转入病弱猪头数、转出猪头数，转出病弱猪头数
#     diminished_data['batchno'] = dic['batchno']
#     diminished_data['age'] = (diminished_data.time - step_2_data.astype({'indate':'datetime64'}).iloc[0,4].date()).apply(lambda x: x.days)
#     diminished_data['age'] += int(round(dic['indayold']))
#     diminished_data['ffieldid'] = dic['ffieldid']
#     diminished_data['unit'] = dic['unit']
#     diminished_data['ingoodqty'] = dic['ingoodqty']
#     diminished_data['inbadqty'] = dic['inbadqty']
#     diminished_data['outqty'] = dic['outqty']
#     diminished_data['outbadqty'] = dic['outbadqty']
#     diminished_data['indayold'] = dic['indayold']
#     diminished_data['outdayold'] = dic['outdayold']
#     diminished_data['staravgweight'] = dic['staravgweight']
#     diminished_data['endavgweight'] = dic['endavgweight']
#     diminished_data = diminished_data[['time','batchno','ffieldid','unit','age','ingoodqty','inbadqty','outqty','outbadqty','indayold','outdayold','staravgweight','endavgweight'] + var_name]
#     return diminished_data 

# step_3_data = []
# for i in range(10):
#     print('current progress: '+ str(i+1)+'/'+ str(step_2_data.shape[0]))
#     try:
#         step_3_data.append(data_mani(dict(step_2_data.iloc[i,:])))
#     except:
#         print('存在环控，但数据异常',dict(step_2_data.iloc[i,:])['id'])
# step_3_data = pd.concat(step_3_data,axis = 0)

# step_3_data.to_csv('result_step3_OCT_beta1.csv',index = False)

## 处方笺提取/时间窗口建立

In [ ]:
# import
import sys
from pyhive import hive
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) #显示所有列
pd.set_option('display.float_format', lambda x: '%.2f' % x)
sys.path.append('/data/datamining/envir/Py_functions')
from get_data.get_data_func import * ##取数function

In [ ]:
## 建立时间窗口
data = pd.read_csv('result_step3_OCT_beta1.csv').astype({'time':'datetime64'})
## 创建unique id 
data['id'] = data.apply(lambda x: str(x.ffieldid)+'/'+str(x.unit) + '/' + str(x.batchno)[:5],axis = 1)
## 删去异常值
data = data[(data.day_temp_max <= 40)
                        & (data.day_temp_min > 10) &
                        (data.night_temp_max <= 40) &
                        (data.night_temp_min > 10) &
                        (data.day_shower_max < 7200) &
                        (data.night_shower_max < 7200)]
## 删去重复值
data = data.drop_duplicates(['time','id']).reset_index(drop = True)
## 环控参数
prop =  ['day_temp_max', 'day_temp_min', 'day_temp_mean','day_temp_std', 
         'night_temp_max', 'night_temp_min', 'night_temp_mean', 'night_temp_std',
         'day_shower_max','day_shower_min', 'day_shower_mean', 'day_shower_std',
       'night_shower_max', 'night_shower_min', 'night_shower_mean','night_shower_std']
## 创建时间窗口
def addsuffix(lst,s):
    res = []
    for i in list(lst):
        res.append(i + s)
    return res
## 前3天时间窗口建立
data_pre1 = data.copy()
data_pre1.time = data.time + dt.timedelta(1)
tm1 = data[['time','id']].merge(data_pre1,how = 'left',on = ['time','id'])[prop]
tm1.columns = addsuffix(tm1[prop].columns,'_tm1')
data_pre2 = data.copy()
data_pre2.time = data.time + dt.timedelta(2)
tm2 = data[['time','id']].merge(data_pre2,how = 'left',on = ['time','id'])[prop]
tm2.columns = addsuffix(tm2[prop].columns,'_tm2')
data_pre3 = data.copy()
data_pre3.time = data.time + dt.timedelta(3)
tm3 = data[['time','id']].merge(data_pre3, how = 'left',on = ['time','id'])[prop]
tm3.columns = addsuffix(tm3[prop].columns,'_tm3')
## concat
data = pd.concat([data,tm1,tm2,tm3],axis = 1)
## 温度变化指标
data['daychange']= data.day_temp_mean - data.day_temp_mean_tm1 ##相比昨天日间温度变化
data['nightchange'] = data.night_temp_mean - data.night_temp_mean_tm1 ##相比昨天夜间温度变化
data['dailychange'] = data.day_temp_max - data.night_temp_min ##昼夜温差
## 绝对值
data['daychange_abs'] = data.daychange.apply(np.abs)
data['nightchange_abs'] = data.nightchange.apply(np.abs)
data['dailychange_abs'] = data.dailychange.apply(np.abs)

## 剔除nan值
data = data.dropna()

## 提取咳嗽标签
batchlist  = tuple(data.batchno.unique())
cough = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
           sql = '''SELECT date(attack_date) AS `date`,batchno,ffieldid,unit_no
                    FROM mydw.source_my_sc_recipe AS re 
                    INNER JOIN mydw.source_my_sc_cfq_appy_sympto neo
                    ON neo.my_recipeid = re.id
                    WHERE segmentid = 13906
                    AND hxd_sympto LIKE '%咳嗽%' AND batchno in ''' + str(batchlist))
cough['cough'] = 1
cough = cough.astype({'unit_no':'int'}).rename({'date':'time','unit_no':'unit'},axis = 1).astype({'time':'datetime64'})

## 提取腹泻标签
diarrhea = query_data(method ='inceptor', username = 'szhuangdancheng', password = '2020@hdc',
           sql = '''SELECT date(attack_date) AS `date`,batchno,ffieldid,unit_no
                    FROM mydw.source_my_sc_recipe AS re 
                    INNER JOIN mydw.source_my_sc_cfq_appy_sympto neo
                    ON neo.my_recipeid = re.id
                    WHERE segmentid = 13906
                    AND fmw_sympto LIKE '%腹泻%' AND batchno in ''' + str(batchlist))
diarrhea['diarrhea'] = 1
diarrhea = diarrhea.astype({'unit_no':'int'}).rename({'date':'time','unit_no':'unit'},axis = 1).astype({'time':'datetime64'})

## 数据融合
final_data = data.merge(cough,how = 'left',on = ['time','batchno','ffieldid','unit'])
final_data = final_data.merge(diarrhea,how = 'left',on = ['time','batchno','ffieldid','unit'])
final_data.diarrhea.fillna(0,inplace = True)
final_data.cough.fillna(0,inplace = True)
final_data.to_csv('final_data10.csv',index = False)

## 数据清洗/EDA/特征工程

In [18]:
# import
import sys
import pandas as pd
import seaborn as sns
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None) #显示所有列bb
# pd.set_option('display.max_rows', None) #显示所有列bb
pd.set_option('display.float_format', lambda x: '%.2f' % x)
def addsuffix(lst,s):
    res = []
    for i in list(lst):
        res.append(i + s)
    return res

### 数据读取
数据情况：
共87659条以天、单元为最小单位的数据，包含1829个单元, 1113个批次, 169个厂区。
时间从2020年8月1日到10月31日。

In [19]:
## 数据读取
df = pd.read_csv('final_data10.csv')
## 时间选取
df = df[df.time > '2020-08-01'].astype({'ffieldid':'str','cough' : 'int', 'diarrhea' : 'int'}).reset_index(drop = True)
## 日龄，猪群数量分段
df.age = (df.age//10)*10
df.inbadqty = np.round((df.inbadqty/(df.ingoodqty+df.inbadqty))*100,1)
df.ingoodqty = df.ingoodqty//100*100 
## 猪群转入指数 = 初均重/转入日龄 ：相同日龄转入，越重的猪越好
df['in_index'] = df.staravgweight/df.indayold

### 变量筛选 

In [20]:
## 字段分类
key = ['time','id']
## 猪群变量
batch_var = ['batchno', 'ffieldid', 'unit', 'age', 'ingoodqty', 'inbadqty',
       'outqty', 'outbadqty', 'indayold', 'outdayold', 'staravgweight','endavgweight','in_index' ]
## 环控变量
env =  ['day_temp_max', 'day_temp_min', 'day_temp_mean','day_temp_std', 
         'night_temp_max', 'night_temp_min', 'night_temp_mean', 'night_temp_std',
         'day_shower_max','day_shower_min', 'day_shower_mean', 'day_shower_std',
       'night_shower_max', 'night_shower_min', 'night_shower_mean','night_shower_std']
# 创建时间窗口
env_tm1 = addsuffix(env,'_tm1')
env_tm2 = addsuffix(env,'_tm2')
env_tm3 = addsuffix(env,'_tm3')
# 变化变量
change = ['daychange','nightchange','dailychange','daychange_abs','nightchange_abs','dailychange_abs']
# 合并
env_var = env.copy()
env_var.extend(env_tm1)
env_var.extend(env_tm2)
env_var.extend(env_tm3)
env_var.extend(change)
## 标签
label = ['cough','diarrhea']

#### 人工筛选、根据定义和相关性

In [21]:
## 根据X,Y之间相关性进行筛选，pearsonr不适用与二分类标签
from scipy.stats import pearsonr
def print_corr(var_name):
    for i in var_name:
        try:
            c,p = pearsonr(df[i],df.cough)[0],pearsonr(df[i],df.cough)[1]
            print(i + ':  Corr: '+ str(c)+ ' p-value: ' + str(p))
        except:
            print(i + ': Not applicable')
# print_corr(env_var)

In [22]:
## 根据变量之间相关性选取变量
# df.corr
## 猪群变量
batch_var_selected = ['ffieldid', 'age', 'ingoodqty', 'inbadqty','indayold', 'staravgweight','in_index']
# df[batch_var_selected].corr()
## 环控变量
prop = ['day_temp_mean','day_temp_std','night_temp_mean','night_temp_std',
                'day_shower_max','day_shower_std','night_shower_max','night_shower_std']
## 时间变量
prop_tm1 = addsuffix(prop,'_tm1')
prop_tm2 = addsuffix(prop,'_tm2')
prop_tm3 = addsuffix(prop,'_tm3')
## 变化变量
change = ['daychange','nightchange','dailychange','daychange_abs','nightchange_abs']
env_var_selected = prop.copy()
env_var_selected.extend(prop_tm1)
env_var_selected.extend(prop_tm2)
env_var_selected.extend(prop_tm3)
env_var_selected.extend(change)
# df[env_var_selected].corr().to_csv('corr2.csv')
val_selected = []
val_selected.extend(batch_var_selected)
val_selected.extend(env_var_selected)
print('筛选后变量剩余： ', len(val_selected))

筛选后变量剩余：  44


#### 基于模型筛选

In [23]:
from sklearn.model_selection import cross_val_score
X = df[val_selected]
y = df.cough

In [7]:
## 基于linearSVC模型筛选
from sklearn.feature_selection import  SelectFromModel
from sklearn.svm import  LinearSVC
mod=LinearSVC(C=0.1, penalty="l1", dual=False).fit(X,y)
selectmod=SelectFromModel(mod, prefit=True)
selectmod.transform(X)
cols = selectmod.get_support(indices = True)
X_selected_svm = X.iloc[:,cols]
print(X_selected_svm.shape)
clf = LinearSVC(C=0.1, penalty="l1", dual=False)
cross_val_score(clf, X_selected_svm, y, cv=5, scoring='f1_weighted')

(87659, 38)


array([0.82738941, 0.82738941, 0.82741782, 0.82733593, 0.8274082 ])

In [7]:
# ## 基于SVC模型筛选 (非线性核，时间较长)
# from sklearn.feature_selection import  SelectFromModel
# from sklearn.svm import SVC
# mod=SVC(C=0.1,kernel = 'rbf',class_weight = 'balanced',max_iter = 10).fit(X,y)
# selectmod=SelectFromModel(mod, prefit=True)
# selectmod.transform(X)
# cols = selectmod.get_support(indices = True)
# X_selected_svm = X.iloc[:,cols]
# print(X_selected_svm.shape)
# clf = SVC(C=0.1,kernel = 'rbf',class_weight = 'balanced',max_iter = 10)
# cross_val_score(clf, X_selected_svm, y, cv=5, scoring='f1_macro')

In [24]:
## 基于GBDT模型筛选
from sklearn.ensemble import  GradientBoostingClassifier
gb=GradientBoostingClassifier()
selectmod = SelectFromModel(gb)
selectmod.fit_transform(X,y)
cols = selectmod.get_support(indices = True)
X_selected_gbdt = X.iloc[:,cols]
print(X_selected_gbdt.shape)
clf = GradientBoostingClassifier()
cross_val_score(clf, X_selected_gbdt, y, cv=5, scoring='f1_weighted')

(87659, 13)


array([0.82741782, 0.82749998, 0.82738941, 0.82733593, 0.82597918])

In [ ]:
## 基于逻辑回归模型筛选
from sklearn.linear_model import  LogisticRegressionCV
lrmodel=LogisticRegressionCV(penalty='l1',solver='liblinear')
selectmod=SelectFromModel(lrmodel,threshold=10)
selectmod.fit(X,y)
selectmod.transform(X)
cols = selectmod.get_support(indices = True)
X_selected = X.iloc[:,cols]
X_selected.shape

In [9]:
## 基于树模型筛选
from sklearn.ensemble import  ExtraTreesClassifier
tree=ExtraTreesClassifier().fit(X,y)
selectmod = SelectFromModel(tree,prefit=True)
selectmod.transform(X)
cols = selectmod.get_support(indices = True)
X_selected_tree = X.iloc[:,cols]
print(X_selected_tree.shape)
clf = ExtraTreesClassifier()
cross_val_score(clf, X_selected_tree, y, cv=5, scoring='f1_weighted')

(87659, 27)


array([0.82785901, 0.82878064, 0.82901369, 0.82826583, 0.83082738])

In [ ]:
## 卡方检验选择
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
## 选择k个与自变量卡方相关系数最高的特征，适用于无序分类变量之间的相关性分析
selector = SelectKBest(chi2, k=3)
## X必须为正数
selector.fit_transform(X.iloc[:,:7], y)
cols = selector.get_support(indices = True)
X_selected_chi2 = X.iloc[:,:7].iloc[:,cols]
X_selected_chi2.head(3)

In [12]:
X_selected_gbdt.columns

Index(['ffieldid', 'age', 'ingoodqty', 'indayold', 'staravgweight', 'in_index',
       'day_temp_mean', 'night_temp_mean', 'day_temp_mean_tm1',
       'day_temp_mean_tm2', 'day_temp_std_tm2', 'nightchange', 'dailychange'],
      dtype='object')

In [15]:
from sklearn import tree # 引入tree模型
import pydotplus  

In [31]:
clf = tree.DecisionTreeClassifier(random_state=0, max_depth = 10, splitter = 'random', min_samples_leaf = 5, class_weight = 'balanced' )# 引入模型
clf = clf.fit(X_selected_gbdt, y) # 训练模型

# 可视化树，输出为pdf格式
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = pydotplus.graph_from_dot_data(dot_data) 
graph.write_pdf("tree.pdf") 
clf = tree.DecisionTreeClassifier(random_state=0, max_depth = 5, splitter = 'random', min_samples_leaf = 5, class_weight = 'balanced' )# 引入模型
cross_val_score(clf, X_selected_gbdt, y, cv=5,scoring='f1_weighted')

array([0.22586126, 0.53320798, 0.4326619 , 0.42151899, 0.51574703])